## FOR COMP90024 ASSIGNMENT 2
### BY TEAM 45:
    - William Chen 1400081
    - Petr Andreev 1375858
    - Rafsan Al Mamun 1407776
    - Ojaswi Dheer 1447227

# Lung Disease Analysis in Victoria
---------------------------------
This notebook explores lung disease data collected from SUDO across all of Australia. The dataset includes mortality rates, income levels, employment status, gender etc for asthma and COPD. The objective of this analysis is to gain insights into the relationship of demographics with lung diseases.